In [1]:
!pip install openai
!pip install langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 849.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 7.4 MB/s eta 0:00:00


In [2]:
AZURE_OPENAI_ENDPOINT="https://othai.openai.azure.com/"
AZURE_OPENAI_API_VERSION="2024-05-01-preview"
AZURE_OPENAI_API_KEY="93956dbb9e1342e1bc52ea03ec55a16e"
AZURE_OPENAI_GPT4O_MODEL_NAME="gpt-4o"
AZURE_OPENAI_GPT4O_DEPLOYMENT_NAME="gpt-4o"

In [3]:
from langchain_community.chat_models import AzureChatOpenAI
llm = AzureChatOpenAI(
      openai_api_version=AZURE_OPENAI_API_VERSION,
      azure_deployment=AZURE_OPENAI_GPT4O_MODEL_NAME,
      temperature=0,
      openai_api_key=AZURE_OPENAI_API_KEY,
      openai_api_base=AZURE_OPENAI_ENDPOINT
    )

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/azure_openai.py:168: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://othai.openai.azure.com/ to https://othai.openai.azure.com/openai.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/azure_openai.py:175: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_

In [4]:
import requests
import json

# Download the data
resp = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json')
data = json.loads(resp.text)  # Load data

# Parse the JSON and preview it
print(type(data), len(data))

def json_print(data):
    print(json.dumps(data, indent=2))

json_print(data[0])

<class 'list'> 10
{
  "Category": "SCIENCE",
  "Question": "This organ removes excess glucose from the blood & stores it as glycogen",
  "Answer": "Liver"
}


In [10]:
!pip install weaviate-client # install weaviate-client


In [15]:
import weaviate

auth_config = weaviate.auth.AuthApiKey(api_key="BlWsiDo1vvaVjlnyRPM5rUj5FxQODmwB3QEx")
client = weaviate.Client(
            url="https://dku4yem7rtqnl6glcodgw.c0.europe-west3.gcp.weaviate.cloud",
            auth_client_secret=auth_config,

            additional_headers={
                "X-Azure-Api-Key": AZURE_OPENAI_API_KEY
            }
            )


In [16]:
class_obj = {
            "class": "Document",
            "vectorizer": "text2vec-openai",
            "moduleConfig": {
                "text2vec-openai": {
                    "resourceName": AZURE_OPENAI_ENDPOINT,
                    "deploymentId": "text-embedding-ada-002"
                }
            },
            "properties":
                [
                    {
                        "dataType": ["string"],
                        "description": "URL of where the content came from",
                        "name": "url",
                        "moduleConfig": {
                            "text2vec-openai": {
                                "skip": True,
                                "vectorizePropertyName": True
                            }
                        }
                    },
                    {
                        "dataType": ["string"],
                        "description": "The actual content",
                        "name": "content",
                        "moduleConfig": {
                            "text2vec-openai": {
                                "skip": False,
                                "vectorizePropertyName": False
                            }
                        }
                    },
                    {
                        "dataType": ["string"],
                        "description": "Type of content - Webpage/Document",
                        "name": "type",
                        "moduleConfig": {
                            "text2vec-openai": {
                                "skip": True,
                                "vectorizePropertyName": True
                            }
                        }
                    },
                     {
                        "dataType": ["int"],
                        "description": "Number of words in the content",
                        "name": "wordCount",
                        "moduleConfig": {
                            "text2vec-openai": {
                                "skip": True,
                                "vectorizePropertyName": True
                            }
                        }
                    }

                ],
            "invertedIndexConfig": {
                "indexNullState": True
            }
        }
client.schema.create_class(class_obj)

In [17]:
json_print(data[0])

{
  "Category": "SCIENCE",
  "Question": "This organ removes excess glucose from the blood & stores it as glycogen",
  "Answer": "Liver"
}


In [19]:
with client.batch.configure(batch_size=5) as batch:
    for i, d in enumerate(data):  # Batch import data

        print(f"importing question: {i+1}")

        properties = {
            "answer": d["Answer"],
            "question": d["Question"],
            "category": d["Category"],
        }

        batch.add_data_object(
            data_object=properties,
            class_name="Question"
        )

importing question: 1
importing question: 2
importing question: 3
importing question: 4
importing question: 5
importing question: 6
importing question: 7
importing question: 8
importing question: 9
importing question: 10


In [25]:
count = client.query.aggregate("Question").with_meta_count().do()
json_print(count)

{
  "data": {
    "Aggregate": {
      "Question": [
        {
          "meta": {
            "count": 10
          }
        }
      ]
    }
  }
}


In [27]:
# write a query to extract the vector for a question
result = (client.query
          .get("Question", ["category", "question", "answer"])
          .with_additional("vector")
          .with_limit(10)
          .do())

json_print(result)

{
  "data": {
    "Get": {
      "Question": [
        {
          "_additional": {
            "vector": []
          },
          "answer": "the atmosphere",
          "category": "SCIENCE",
          "question": "Changes in the tropospheric layer of this are what gives us weather"
        },
        {
          "_additional": {
            "vector": []
          },
          "answer": "Antelope",
          "category": "ANIMALS",
          "question": "Weighing around a ton, the eland is the largest species of this animal in Africa"
        },
        {
          "_additional": {
            "vector": []
          },
          "answer": "Liver",
          "category": "SCIENCE",
          "question": "This organ removes excess glucose from the blood & stores it as glycogen"
        },
        {
          "_additional": {
            "vector": []
          },
          "answer": "wire",
          "category": "SCIENCE",
          "question": "A metal that is ductile can be pulled into t

In [29]:
response = (
    client.query
    .get("Question",["question","answer","category"])
    .with_near_text({"concepts": ["biology"]})
    .with_additional('distance')
    .with_limit(2)
    .do()
)

json_print(response)


{
  "errors": [
    {
      "locations": [
        {
          "column": 24,
          "line": 1
        }
      ],
      "message": "Unknown argument \"nearText\" on field \"Question\" of type \"GetObjectsObj\". Did you mean \"nearObject\" or \"nearVector\"?",
      "path": null
    }
  ]
}
